In [53]:
import json
import pandas as pd
import os
from tqdm import tqdm

In [54]:
df_list = []
corpus_files = os.listdir("../corpus/")
for file in corpus_files:
    with open(f"../corpus/{file}", "r", encoding="utf8") as f:
        d = json.load(f)
        df = pd.DataFrame(d)
        df = df.drop(columns=["author", "webpath", "title"])
        df["filepath"] = df["filepath"].apply(lambda x: x.replace("/mnt/c/Users/Moritz Lahann/Desktop/STUDIUM/Module IAS/Master's Thesis/gutenberg-edition16/", ""))
        df_list.append(df)

df = pd.concat(df_list)
df.head()


,genre,filepath,chapters
0,Historische Romane und Erzählungen,schulzsm/moormars/moormars.html,"{'name': 'Erster Teil.', 'idx': 0, 'paragraphs..."
1,Historische Romane und Erzählungen,schulzsm/moormars/moormars.html,"{'name': 'Erstes Kapitel.', 'idx': 1, 'paragra..."
2,Historische Romane und Erzählungen,schulzsm/moormars/moormars.html,"{'name': 'Zweites Kapitel.', 'idx': 2, 'paragr..."
3,Historische Romane und Erzählungen,schulzsm/moormars/moormars.html,"{'name': 'Drittes Kapitel.', 'idx': 3, 'paragr..."
4,Historische Romane und Erzählungen,schulzsm/moormars/moormars.html,"{'name': 'Viertes Kapitel.', 'idx': 4, 'paragr..."


In [55]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize

nltk.download("punkt")

grouped_df = df.groupby(by=["filepath"])
books_df_list = []
for name, group in tqdm(grouped_df):
    for chapter in group["chapters"]:
        paragraph_count = len(chapter["paragraphs"])
        word_count = sum([len(word_tokenize(p, language="german")) for p in chapter["paragraphs"]])
        sentence_count = sum([len(sent_tokenize(p, language="german")) for p in chapter["paragraphs"]])
        book = name

        books_df_list.append(pd.DataFrame({"paragraph_count": [paragraph_count], "word_count": [word_count], "sentence_count": [sentence_count], "book": [book]}))

stats_df = pd.concat(books_df_list)

print(stats_df)

[nltk_data] Downloading package punkt to /Users/mlahann/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
100%|██████████| 5940/5940 [50:27<00:00,  1.96it/s]  


    paragraph_count  word_count  sentence_count                          book
0                 3         423              14  about/badenbad/badenbad.html
0                17        3448             133  about/badenbad/badenbad.html
0                38        3951             153  about/badenbad/badenbad.html
0                33        3558             144  about/badenbad/badenbad.html
0                44        3774             170  about/badenbad/badenbad.html
..              ...         ...             ...                           ...
0                47        4669             141  zznafer/husaren/husaren.html
0               124        9729             383  zznafer/husaren/husaren.html
0                47        4635             201  zznafer/husaren/husaren.html
0                59        3620             154  zznafer/husaren/husaren.html
0               112        7763             314  zznafer/husaren/husaren.html

[145942 rows x 4 columns]


In [56]:
for name, group in stats_df.groupby(by="book"):
    print(name)
    print(len(group))

about/badenbad/badenbad.html
17
about/bergkoen/bergkoen.html
10
about/parisehe/parisehe.html
13
abraham/fabeln/fabeln.html
5
abraham/fabparab/fabparab.html
55
abraham/misc/misc.html
19
abraham/narrnest/narrnest.html
12
achleitn/bergen/bergen.html
68
achleitn/bezirks1/bezirks1.html
5
achleitn/bezirks2/bezirks2.html
12
achleitn/dobratsc/dobratsc.html
10
achleitn/finanzer/finanzer.html
7
achleitn/moor/moor.html
14
adlerrev/japan/japan.html
49
adlersfe/amoenen/amoenen.html
9
adlersfe/margarit/margarit.html
10
adlersfe/weissro1/weissro1.html
2
adolph/frueher/frueher.html
15
adolph/haus37/haus37.html
24
adolph/schacker/schacker.html
9
adolph/toechter/toechter.html
16
aesop/fabeln/fabeln.html
80
aesop/fabeln1/fabeln1.html
50
aesop/fabeln2/fabeln2.html
114
ahlborn/favorite/favorite.html
33
aho/blut/blut.html
17
aho/einsam/einsam.html
6
aho/geaecht/geaecht.html
9
aimard/freikuge/freikuge.html
28
aimard/mexnaec1/mexnaec1.html
10
aimard/mexnaec2/mexnaec2.html
9
aimard/mexnaec3/mexnaec3.html
10
ai

In [57]:
import numpy as np

print("no. books", len(stats_df["book"].unique()))
print("no. words", stats_df["word_count"].sum())
print("no. paragraphs", stats_df["paragraph_count"].sum())
print("no. sentences", stats_df["sentence_count"].sum())
print("no. chapters", len(stats_df))
print("avg paragraph count", stats_df["paragraph_count"].mean())
print("median paragraph count", stats_df["paragraph_count"].median())
print("avg word count", stats_df["word_count"].mean())
print("median word count", stats_df["word_count"].median())
print("avg sentence count", stats_df["sentence_count"].mean())
print("median sentence count", stats_df["sentence_count"].median())
print("avg chapter count", np.mean([len(b) for b in stats_df.groupby(by="book").groups.values()]))
print("median chapter count", np.median([len(b) for b in stats_df.groupby(by="book").groups.values()]))
print("no. chapter breaks", sum([len(b) - 1 for b in stats_df.groupby(by="book").groups.values()]))

no. books 5940
no. words 464271725
no. paragraphs 7864259
no. sentences 23269276
no. chapters 145942
avg paragraph count 53.8861945156295
median paragraph count 32.0
avg word count 3181.2070891175945


median word count 2164.0
avg sentence count 159.4419426895616
median sentence count 104.0
avg chapter count 24.56936026936027
median chapter count 17.0
no. chapter breaks 140002
